In [1]:
from utils.sampling import *
from tools import builder
from datasets.data_transforms import ResamplePoints
import open3d as o3d
import numpy as np
import glob
from utils.vis_utils import convert_to_o3dpcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import torch 

model = builder.model_builder({'NAME':'PCN'})
model.cuda()

ckpt_path = 'demo_data/R6D-E3_frustum_norm_fps_coarse_lidarsim_surface_knnmeshsurface_50ep.pth'
state_dict = torch.load(ckpt_path, map_location=f'cuda:0')
base_ckpt = {k.replace("module.", ""): v for k, v in state_dict['base_model'].items()}
model.load_state_dict(base_ckpt)

model.eval()

PCN(
  (pose_encoder): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (5): AdaptiveMaxPool1d(output_size=1)
  )
  (pose_fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=9, bias=True)
  )
  (encoder): FeatureEncoder(
    (mlp_conv1): Sequential(
      (0): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (mlp_conv2): Sequential(
      (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [6]:
pcd_dir = 'demo_data/temp'
pcd_data = glob.glob(pcd_dir + '/*')
idx = 6
raw = np.fromfile(pcd_data[idx], dtype=np.float32).reshape((-1, 4))[:,:3]
dbpcd = convert_to_o3dpcd(raw)
o3d.visualization.draw_geometries([dbpcd])

In [7]:
eps = 0.7
# dist = np.linalg.norm(np.mean(inpc, axis=0))
labels = np.array(dbpcd.cluster_dbscan(eps=eps, min_points=3, print_progress=False))
y = np.bincount(labels[labels >= 0])
if len(y) != 0:
    value = np.argmax(y)
    most_points = np.argwhere(labels == value)
    f_pcd = dbpcd.select_by_index(most_points)
    
inpc = np.asarray(f_pcd.points)
resample = ResamplePoints({'n_points': 1024})

in_dict = {}
in_dict['input'] = torch.from_numpy(resample(inpc)).float().cuda().unsqueeze(0)

ret_dict = model(in_dict)
pred_surface = get_partial_mesh( in_dict['input'].squeeze(0), ret_dict['coarse'].squeeze(0), k=30)
output = get_largest_cluster(pred_surface, eps=0.4, min_points=1)

In [ ]:
vis_list = []

vis_list.append(convert_to_o3dpcd(in_dict['input'].squeeze(0).cpu().numpy(), [0.14, 0.34, 0.69]))
vis_list.append(convert_to_o3dpcd(ret_dict['coarse'].squeeze(0).detach().cpu().numpy(), [0.70, 0.85, 0.94]))
vis_list.append(convert_to_o3dpcd(output.detach().cpu().numpy()))

o3d.visualization.draw_geometries(vis_list)

In [4]:
pcd_dir = 'demo_data'
pcd_data = glob.glob(pcd_dir + '/*')
idx = 0

inpc = np.asarray(o3d.io.read_point_cloud(pcd_data[idx]).points)
resample = ResamplePoints({'n_points': 1024})
print('Num input pts: ', inpc.shape[0])

in_dict = {}
in_dict['input'] = torch.from_numpy(resample(inpc)).float().cuda().unsqueeze(0)

ret_dict = model(in_dict)
pred_surface = get_partial_mesh( in_dict['input'].squeeze(0), ret_dict['coarse'].squeeze(0), k=30)
output = get_largest_cluster(pred_surface, eps=0.4, min_points=1)

Num input pts:  91
